# JSON exercise

Download the data from [**here**](https://drive.google.com/file/d/1DGaX5AVfYhmWeb15lI-MzUbSKTYSz9fQ/view?usp=sharing) and answer following questions:
1. Find the 10 countries with most projects
2. What are top 10 sectors with projects?
3. Find the top 10 major project themes (using column 'mjtheme_namecode')
4. In 3. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [260]:
# open json file 

import pandas as pd
import json
#load json object
with open('world_bank_projects.json') as f:
    world_bank_nested_json = json.load(f)

### 1. Find the 10 countries with most projects

In [261]:
# creating a normilized json file
normalized_json=pd.json_normalize(world_bank_nested_json)

#assuming that each line is a unique project aggregating lines by country names
#create anew column for counted countries, sort by that column (descending) and return only top ten
grouped_by_countryname=normalized_json.groupby('countryname').size().reset_index(name="CountCountries").sort_values(by='CountCountries',ascending=False).head(10)
grouped_by_countryname

,countryname,CountCountries
39,People's Republic of China,19
64,Republic of Indonesia,19
107,Socialist Republic of Vietnam,17
63,Republic of India,16
97,Republic of Yemen,13
38,People's Republic of Bangladesh,12
34,Nepal,12
25,Kingdom of Morocco,12
76,Republic of Mozambique,11
0,Africa,11


### 2. What are top 10 sectors with projects?

In [263]:
# creating a normilized json file for 'sector' column which is a list of services
normalized_sector=pd.json_normalize(world_bank_nested_json,record_path='sector')

#create a new column for counted Names (Service types), sort by that column (descending) and return only top ten
grouped_by_services=normalized_sector.groupby('Name').size().reset_index(name="Count").sort_values(by='Count',ascending=False).head(10)
grouped_by_services


,Name,Count
40,Other social services,106
7,Central government administration,82
61,Sub-national government administration,75
15,"General agriculture, fishing and forestry sector",65
24,Health,63
21,General public administration sector,51
56,Rural and Inter-Urban Roads and Highways,44
23,"General water, sanitation and flood protection...",39
1,Agricultural extension and research,38
45,"Public administration- Agriculture, fishing an...",36


### 3. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [373]:
# creating a normilized json file for 'mjtheme_namecode' column which is a list of themes' codes and their names
normalized_mjtheme=pd.json_normalize(world_bank_nested_json,record_path='mjtheme_namecode')

#create a new column for counted codes of themes, sort by that column (descending) and return only top ten
grouped_by_themes=normalized_mjtheme.groupby('code').size().reset_index(name='CountThemeCodes').sort_values('CountThemeCodes',ascending=False).head(10)
grouped_by_themes

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
...,...,...
1494,10,Rural development
1495,9,Urban development
1496,8,Human development
1497,5,Trade and integration


### 4. In 3. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [370]:
normalized_mjtheme

# checking if empty cells in rows have Nan values or have spaces
empty_or_space_mask = normalized_mjtheme['name'].str.strip().eq('') | normalized_mjtheme['name'].isna()

#filtering by the mask and dropping duplicates to get a table with unique codes and theme names
filtered_mjtheme_no_duplicates = normalized_mjtheme[~empty_or_space_mask].drop_duplicates()

# Merge and update values based on 'code' column (creating an additional temp column "name_new")
updated_mjtheme = normalized_mjtheme.merge(
    filtered_mjtheme_no_duplicates, on='code', how='left', suffixes=('', '_new')
)

# Update 'name' column with values from 'name_new'
updated_mjtheme['name'] = updated_mjtheme['name_new']

# Drop the temporary column 'name_new'
updated_mjtheme = updated_mjtheme.drop(['name_new'], axis=1)

# Print the updated DataFrame
print(updated_mjtheme)


     code                                          name
0       8                             Human development
1      11  Environment and natural resources management
2       1                           Economic management
3       6         Social protection and risk management
4       5                         Trade and integration
...   ...                                           ...
1494   10                             Rural development
1495    9                             Urban development
1496    8                             Human development
1497    5                         Trade and integration
1498    4      Financial and private sector development

[1499 rows x 2 columns]


### Another approach where by mistake was using 'mjthemecode' column consisting of codes in string type with different.Longer and difficult way but good to compare results and learn something new
### 

In [281]:
#selecting all rows and 'mjthemecode' column from 'normalized_json'
x = normalized_json.loc[:,['mjthemecode']]

#  creating a string with all values from  'mjthemecode'
joined_string = ','.join(normalized_json['mjthemecode'])

# Split the joined string into individual values (creating a list)
individual_values = joined_string.split(',')

# counting individual values using Counter function and printing the top ten
from collections import Counter
themes_count=Counter(individual_values)
themes_count.most_common(10)


[('11', 250),
 ('10', 216),
 ('8', 210),
 ('2', 199),
 ('6', 168),
 ('4', 146),
 ('7', 130),
 ('5', 77),
 ('9', 50),
 ('1', 38)]

### Wrong approaches below (don't use)

In [257]:
#adding a "Projects_Count" column where aggregate number
normalized_json['Projects_Count']=normalized_json.count(axis='columns')
sorted_projects = normalized_json.sort_values(by='Projects_Count',ascending=False).head(10)
countries = sorted_projects.loc[sorted_projects.index,['countryname','Projects_Count']]
#print(normalized_json)
print(countries)

                                 countryname  Projects_Count
0    Federal Democratic Republic of Ethiopia              55
213         Democratic Republic of the Congo              55
185                        Republic of Haiti              55
186                        Republic of Haiti              55
199            Socialist Republic of Vietnam              55
204                       Republic of Zambia              55
207                       Republic of Uganda              55
211                          Kyrgyz Republic              55
215                        Republic of Niger              55
253                  Republic of South Sudan              55
